In [17]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from torchvision.datasets import CocoDetection

from architectures.basic import BoundingBoxModel as BasicBoundingBoxModel
from architectures.bigger_basic import BoundingBoxModel as BiggerBasicBoundingBoxModel
from functions.mapping import extract_bboxes

print(torch.__version__)
print(torch.cuda.is_available())

2.3.1
True


In [10]:
root = os.path.join("data", "test_images")
ann_file = os.path.join("data", "test_coco_annotations.json")
transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = CocoDetection(
    root=root,
    annFile=ann_file,
    transform=transform
)
test_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    prefetch_factor=16
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
cuda


In [11]:
basic_model = BasicBoundingBoxModel().to(device)
bigger_basic_model = BiggerBasicBoundingBoxModel().to(device)
bigger_basic_ciou_model = BiggerBasicBoundingBoxModel().to(device)

In [12]:
basic_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_1_basic.pt"), map_location=device)["model_state_dict"])
basic_model.eval()
bigger_basic_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_2_bigger_basic.pt"), map_location=device)["model_state_dict"])
bigger_basic_model.eval()
bigger_basic_ciou_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_3_bigger_basic_ciou.pt"), map_location=device)["model_state_dict"])
bigger_basic_ciou_model.eval()

BoundingBoxModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=57600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [31]:
def test_model(model: nn.Module) -> tuple[torch.Tensor, torch.Tensor]:
    results = []
    all_targets = []

    with torch.no_grad():
        for images, targets in test_loader:
            images = images.to(device)
            outputs = model(images)
            
            results.extend(outputs)
            all_targets.extend(extract_bboxes(targets))
            
    return torch.stack(results).to(device), torch.stack(all_targets).to(device)

In [32]:
basic_model_results, all_targets = test_model(basic_model)
bigger_basic_model_results, _ = test_model(bigger_basic_model)
bigger_basic_ciou_model_results, _ = test_model(bigger_basic_ciou_model)

In [37]:
basic_loss = nn.SmoothL1Loss()(basic_model_results, all_targets)
bigger_basic_loss = nn.SmoothL1Loss()(bigger_basic_model_results, all_targets)
bigger_basic_ciou_loss = nn.SmoothL1Loss()(bigger_basic_ciou_model_results, all_targets)

print(basic_loss)
print(bigger_basic_loss)
print(bigger_basic_ciou_loss)

tensor(6.0493, device='cuda:0')
tensor(6.1762, device='cuda:0')
tensor(9.2023, device='cuda:0')
